In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
# import pyspark
import json
import ast
import boto3
from sagemaker import get_execution_role
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from IPython.display import clear_output

In [2]:
role = get_execution_role()
bucket='sagemaker-us-east-1-770526846351'

IMPORT INVESTORS DATA FROM s3

In [3]:
# data_key = 'investor.csv'
# data_location = 's3://{}/{}'.format(bucket, data_key)
# df_investor = pd.read_csv(data_location)

DATA CLEANING/IMPUTATION

In [4]:
# def unpack(dict_str, key):
#     try:
#         as_dict = ast.literal_eval(dict_str)
#     except:
#         return np.nan
#     if not key in as_dict.keys():
#         return np.nan
#     return as_dict[key]

In [5]:
# # extract values from situs dict

# df_investor['zip'] = df_investor['situs'].apply(lambda x: unpack(x, 'zip'))
# df_investor['state'] = df_investor['situs'].apply(lambda x: unpack(x, 'state'))

In [6]:
# # drop rows where state, zip, buyerId, propertyType is null & not worth imputing.

# df_investor = df_investor[df_investor['state'].notna()]
# df_investor = df_investor[df_investor['zip'].notna()]
# df_investor = df_investor[df_investor['buyerId'].notna()]
# df_investor = df_investor[df_investor['propertyType'].notna()]

In [7]:
# # check nulls

# df_investor.isnull().sum()[df_investor.isnull().sum() > 0].sort_values(ascending=False)

purchasePrice    895427
sqft             486935
value             18627
purchaseDate        550
dtype: int64

In [8]:
# # calculate median of purchasePrice for non-null records

# df_investor_pp = df_investor[df_investor['purchasePrice'].notna()]
# pp = df_investor_pp.groupby(['zip', 'propertyType'])['purchasePrice'].median().reset_index(name='med_pp')

In [9]:
# # calculate median of sqft for non-null records

# df_investor_sqft = df_investor[df_investor['sqft'].notna()]
# sqft = df_investor_sqft.groupby(['zip', 'propertyType'])['sqft'].median().reset_index(name='med_sqft')

In [10]:
# # calculate median of value for non-null records

# df_investor_val = df_investor[df_investor['value'].notna()]
# val = df_investor_val.groupby(['zip', 'propertyType'])['value'].median().reset_index(name='med_val')

In [11]:
# # join medians back to original df

# dfs = [df_investor, pp, sqft, val]

# df_investor = reduce(lambda left,right: pd.merge(left,right,on=['zip', 'propertyType'], how='inner'), dfs)

In [13]:
# # replace null purchasePrice, sqft, value with median values

# df_investor['purchasePrice'] = df_investor['purchasePrice'].fillna(df_investor['med_pp'])
# df_investor['sqft'] = df_investor['sqft'].fillna(df_investor['med_sqft'])
# df_investor['value'] = df_investor['value'].fillna(df_investor['med_val'])

In [14]:
# df_investor['pricePerSqft'] = df_investor['purchasePrice']/df_investor['sqft']
# df_investor['valuePerSqft'] = df_investor['value']/df_investor['sqft']

In [15]:
# # get list of investors with >= 10 transactions (filtering out small transactions + addressing cold start problem)

# investors = df_investor.groupby(['buyerId'])['buyerId'].count().reset_index(name='txn_count').sort_values(['txn_count'], ascending=False) 
# investors = investors[investors['txn_count'] >= 10] 

In [16]:
# # filter df_investors to include investor swith >= 10 transactions

# base = pd.merge(df_investor, investors, on=['buyerId'], how='inner')

In [2]:
# base['state'].value_counts()

In [17]:
# # save to csvs

# investors.to_csv('s3://{}/{}'.format(bucket, 'investor_txns.csv'))

# base.to_csv('s3://{}/{}'.format(bucket, 'investor_filtered_cleaned.csv'))

LOAD IN CLEANED INVESTORS DATA FROM s3

In [18]:
# # read in cleaned df_investor data from s3 to ease memory usage

# df_investor_location = 's3://{}/{}'.format(bucket, 'investor_filtered_cleaned.csv')

# df_investor = pd.read_csv(df_investor_location)

SPLIT DATA INTO STRATIFIED TRAIN/TEST

In [19]:
# # create 10% stratified sample based off 'state', 'propertyType', 'fundingSource', 'investmentType'

# df_investor_test = df_investor \
# .groupby(['state', 'propertyType', 'fundingSource', 'investmentType'], group_keys=False) \
# .apply(lambda x: x.sample(frac=0.1))

In [20]:
# # set test group to 1

# df_investor_test['test'] = 1

In [21]:
# # left join to identify train/test

# df_investor = pd.merge(df_investor, df_investor_test[['id', 'test']], on=['id'], how='left')

In [22]:
# # set train group as 0

# df_investor['test'] = df_investor['test'].fillna(0)

In [23]:
# # drop unnecessary columns

# df_investor = df_investor.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [24]:
# # split df_investor out into train/test

# train = df_investor[df_investor['test'] == 0]
# test = df_investor[df_investor['test'] == 1]

In [25]:
# # write train/test sets to s3 as csv

# train.to_csv('s3://{}/{}'.format(bucket, 'train.csv'))
# test.to_csv('s3://{}/{}'.format(bucket, 'test.csv'))

CREATE MEDIAN MAPPINGS

In [26]:
# med_pps = df_investor.groupby(['buyerId'])['pricePerSqft'].median().reset_index(name='med_pps')
# med_vps = df_investor.groupby(['buyerId'])['valuePerSqft'].median().reset_index(name='med_vps')
# med_sqft = df_investor.groupby(['buyerId'])['sqft'].median().reset_index(name='med_sqft')
# med_val = df_investor.groupby(['buyerId'])['value'].median().reset_index(name='med_val')
# med_pp = df_investor.groupby(['buyerId'])['purchasePrice'].median().reset_index(name='med_pp')

# med_pps.to_csv('s3://{}/{}'.format(bucket, 'med_pps.csv'))
# med_vps.to_csv('s3://{}/{}'.format(bucket, 'med_vps.csv'))
# med_sqft.to_csv('s3://{}/{}'.format(bucket, 'med_sqft.csv'))
# med_val.to_csv('s3://{}/{}'.format(bucket, 'med_val.csv'))
# med_pp.to_csv('s3://{}/{}'.format(bucket, 'med_pp.csv'))

LOAD IN TRAIN/TEST DATA FROM s3

In [27]:
train_location = 's3://{}/{}'.format(bucket, 'train.csv')
test_location = 's3://{}/{}'.format(bucket, 'test.csv')

df_train = pd.read_csv(train_location)
df_test = pd.read_csv(test_location)

In [40]:
df_investor = pd.concat([df_train, df_test])

TRANSACTION SIMILARITY TRANSFORMATIONS

In [18]:
# df_investor['state'].value_counts()

In [36]:
df_investor[df_investor['state']=='DC']

,Unnamed: 0,id,buyerId,importDate,taxId,mailId,name,hasPhone,hasEmail,isEntity,...,propertyType,zip,state,med_pp,med_sqft,med_val,pricePerSqft,valuePerSqft,txn_count,test
19225,21421,11001-20220819-308-1,c08e119533785c7ab76fe7aaf36a9523,20220824,1-11001-5869 1134,9983978732a12527af9bace4ca3227a8,EK REAL ESTATE FUND I LLC,True,False,True,...,Residential,20020,DC,235000.0,1024.0,345595.0,370.192308,393.020833,209,0.0
19861,22127,11001-20181126-212-0,18d85db28e29dbb982592799024e2d7a,20190410,1-11001-5084 2012,b76c230eb72acb5f7dac62436b897a87,MTGLQ INVTRS LP,True,False,True,...,Residential,20019,DC,230000.0,1024.0,388430.0,101.049869,177.440945,168,0.0
29020,32305,11001-20190513-64-0,c9bf824511a49c2ddd62d03e9d2c35e7,20200212,1-11001-5292 0082,185f5bfcf650ca090da454531c1a2896,HILLDALE TRUST,True,False,False,...,Residential,20019,DC,230000.0,1024.0,388430.0,230.230000,427.062500,28,0.0
29304,32620,11001-20210920-371-0,d4bdfc75dbd54acaa38f3aeb2471f142,20210922,1-11001-5538 0081,e8aaa1c5de269060f28858d8a03c3bf9,EK REAL ESTATE FUND I LLC,True,False,True,...,Residential,20020,DC,235000.0,1024.0,345595.0,549.450549,458.360806,196,0.0
70032,77829,11001-20180621-264-0,18d6b55172d43bba8b7a19cf64caa3fa,20201125,1-11001-5881 0050,de919b76f429197b802d4e3eb087e57d,DIPLOMAT PROPERTY MANAGER LLC,True,False,True,...,Residential,20020,DC,235000.0,1024.0,345595.0,139.000000,276.316176,26,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52906,528680,11001-20200111-504-0,c5ba940f6cbc5cb5e8339c541d4a13c1,20201021,1-11001-2868 0080,e26b7500b88c63559f7b0ba2c7ed742c,DILAN INVESTMENT LLC,True,False,True,...,Residential,20009,DC,670079.0,988.0,656770.0,541.666667,755.721726,81,1.0
58468,583922,11001-20190306-25-1,928832f83eadde1351e687f01e7010d1,20200610,1-11001-3122 0019,e51610c07d83f70188174967beb3d84f,DOGWOOD RESTORATION LLC,True,False,True,...,Residential,20001,DC,656000.0,1479.0,867945.0,291.149068,618.454969,27,1.0
58469,583931,11001-20180221-21-1,928832f83eadde1351e687f01e7010d1,20190109,1-11001-3116 0041,e51610c07d83f70188174967beb3d84f,DOGWOOD RESTORATION LLC,True,False,True,...,Residential,20001,DC,656000.0,1479.0,867945.0,572.100313,936.065831,27,1.0
58654,585566,11001-20220215-2-1,90221dee94e7774e52ea96daaf6c48db,20220223,1-11001-4325 2047,6d5f4979af432637311eff79fea06eb7,1727 MASS AVE LLC,True,False,True,...,Residential,20018,DC,399999.0,1405.5,637630.0,290.275036,255.021771,10,1.0


In [40]:
# filter features

df_txn = df_investor[['id', 'sqft', 'value', 'purchasePrice', 'propertyType', 'pricePerSqft', 
                     'valuePerSqft', 'zip', 'state']]

In [41]:
# create dummy vars

dummies = ['state', 'propertyType']


for col in dummies:
    vals = df_txn[col].unique().tolist()
    df_txn[col] = pd.Categorical(df_txn[col], categories=vals)
    

df_txn = pd.get_dummies(df_txn, columns=dummies, drop_first=True)


clear_output()

In [42]:
# set index

df_txn = df_txn.set_index('id')

In [43]:
# scale inputs

scaler_prop = StandardScaler()
txn_scaled = scaler_prop.fit_transform(df_txn)

In [47]:
# dimensionality reduction

txn_pca = PCA(n_components=8)
txn_scaled_pca = txn_pca.fit_transform(txn_scaled)

In [48]:
# Final df

df_txn = pd.DataFrame(txn_scaled_pca, index=df_txn.index) #columns=df_txn.columns

In [89]:
df_txn.head()

,0,1,2,3,4,5,6,7
id,,,,,,,,
39113-20210226-3644-0,-0.301298,-0.546314,-0.104104,0.066825,-0.007681,0.039699,0.020044,-0.011568
39113-20210327-725-0,-0.303966,-0.560204,-0.100781,0.074098,-0.017417,0.046923,0.024052,-0.010863
39113-20220504-71-0,-0.212017,-0.154348,-0.122105,-0.015650,0.127444,-0.033579,-0.033286,-0.016304
39113-20210626-514-0,-0.306268,-0.567259,-0.106608,0.069344,-0.012742,0.041548,0.021907,-0.011514
39113-20220725-450-0,-0.199218,-0.102462,-0.115829,-0.018570,0.136025,-0.036113,-0.036573,-0.016369


In [50]:
df_txn.shape

(678566, 8)

In [90]:
df_txn.to_csv('s3://{}/{}'.format(bucket, 'transaction_sim_input.csv'))

INVESTOR SIMILARITY TRANSFORMATIONS

In [54]:
# read in buyerId to total transaction count mapping

investors = 's3://{}/{}'.format(bucket, 'investor_txns.csv')
investors = pd.read_csv(investors)

In [55]:
# read in median mappings

med_pps = 's3://{}/{}'.format(bucket, 'med_pps.csv')
med_vps = 's3://{}/{}'.format(bucket, 'med_vps.csv')
med_sqft = 's3://{}/{}'.format(bucket, 'med_sqft.csv')
med_val = 's3://{}/{}'.format(bucket, 'med_val.csv')
med_pp = 's3://{}/{}'.format(bucket, 'med_pp.csv')

med_pps_df = pd.read_csv(med_pps)
med_vps_df = pd.read_csv(med_vps)
med_sqft_df = pd.read_csv(med_sqft)
med_val_df = pd.read_csv(med_val)
med_pp_df = pd.read_csv(med_pp)

In [56]:
# aggregate for transaction counts

df_train_agg = df_train \
.groupby(['buyerId', 'investmentType', 'fundingSource', 'propertyType', 'state'])['taxId'] \
.count().reset_index(name='cnt')

In [57]:
# add in total transaction counts to create normalized proportions

df_train_agg = pd.merge(df_train_agg, investors, on=['buyerId'], how='inner')
df_train_agg['norm_cnt'] = df_train_agg['cnt']/df_train_agg['txn_count']

df_train_agg.head()

,buyerId,investmentType,fundingSource,propertyType,state,cnt,Unnamed: 0,txn_count,norm_cnt
0,00074b1019380cc21471e2a46825c1fe,Landlord,Cash,Residential,PA,1,254,13,0.076923
1,00074b1019380cc21471e2a46825c1fe,Landlord,Mortgage,Commercial,PA,1,254,13,0.076923
2,00074b1019380cc21471e2a46825c1fe,Rehabber,Cash,Commercial,PA,1,254,13,0.076923
3,00074b1019380cc21471e2a46825c1fe,Rehabber,Cash,Residential,PA,8,254,13,0.615385
4,00074b1019380cc21471e2a46825c1fe,Rehabber,Mortgage,Residential,PA,1,254,13,0.076923


In [58]:
# create composite of 'investmentType', 'fundingSource', 'propertyType', 'state'

df_train_agg['composite'] = df_train_agg[['investmentType', 'fundingSource', 'propertyType', 'state']].agg('-'.join, axis=1)

In [79]:
# pivot data on buyerId

inv_pivot = df_train_agg.pivot(index = 'buyerId', columns ='composite', values = 'norm_cnt')

In [80]:
# add in medians by buyerId

dfs = [med_pps_df[['buyerId', 'med_pps']], med_vps_df[['buyerId', 'med_vps']], med_sqft_df[['buyerId', 'med_sqft']], 
       med_val_df[['buyerId', 'med_val']], med_pp_df[['buyerId', 'med_pp']], inv_pivot]

inv_pivot = reduce(lambda left,right: pd.merge(left,right,on=['buyerId'], how='inner'), dfs)
inv_pivot = inv_pivot.set_index('buyerId').fillna(0)

In [84]:
# scale inputs

inv_scaler = StandardScaler()
inv_pivot_scaled = inv_scaler.fit_transform(inv_pivot)

In [85]:
# dimensionality reduction

inv_pca = PCA(n_components=20)
inv_scaled_pca = inv_pca.fit_transform(inv_pivot_scaled)

In [86]:
# final df

inv_pivot = pd.DataFrame(inv_scaled_pca, index=inv_pivot.index)

In [87]:
inv_pivot.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
buyerId,,,,,,,,,,,,,,,,,,,,
00074b1019380cc21471e2a46825c1fe,-0.094402,-0.065866,-0.177974,0.003851,-0.009422,-0.143129,-0.208115,-0.032254,0.149846,0.165633,0.154581,-0.630407,-0.035712,-0.190031,0.022532,-0.623338,-0.225666,0.154971,0.834998,-1.660746
0007fba1809a022de16900e96add9515,-0.108221,-0.044397,-0.044844,-0.002445,-0.055238,0.002367,-0.059136,-0.123322,-0.156012,0.301529,0.283052,-0.394772,0.060220,-0.357388,0.471606,-0.006292,0.512660,-0.110004,0.343123,0.292158
0015cc309601fe8731febcc98fc7f6c2,-0.140629,-0.047382,-0.235733,0.003086,-0.055420,-0.099428,-0.303354,-0.313086,-0.245393,0.407382,0.433072,-0.434736,-0.043284,-0.895177,-0.075790,0.111116,-0.065325,-0.043298,0.335285,0.186156
001a5216b4a83da8dd43f76628258482,0.099706,0.049352,0.587093,-0.150350,0.010570,0.009603,0.547223,0.548510,0.269935,-1.358669,-1.054707,0.932636,-0.431525,1.553163,-0.263420,-0.057401,-0.063983,0.054078,-0.077545,-0.141645
001bcb3fd35ccd4829fe2ebc7caeb142,-0.080162,-0.022707,-0.017690,-0.044075,-0.027772,-0.030912,-0.013000,-0.049489,-0.037597,-0.121012,-0.070373,-0.021230,-0.092670,0.012817,-0.054513,0.012453,0.116511,-0.167184,0.102567,-0.094450


In [88]:
inv_pivot.shape

(17650, 20)

In [91]:
inv_pivot.to_csv('s3://{}/{}'.format(bucket, 'investor_sim_input.csv'))